In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Duplicate key in file PosixPath('/opt/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 250 ('font.family : IPAexGothic')


In [47]:
train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("test.tsv", sep="\t")
sample = pd.read_csv("sample_submit.csv", header=None)

In [4]:
train.head()

id  Y cap-shape cap-surface cap-color bruises odor gill-attachment  \
0   0  p         f           y         n       f    f               f   
1   2  p         f           y         y       f    f               f   
2   3  e         b           y         w       t    l               f   
3   5  p         x           s         b       t    f               f   
4   7  p         x           s         w       t    f               f   

  gill-spacing gill-size  ... stalk-surface-below-ring stalk-color-above-ring  \
0            c         n  ...                        k                      w   
1            c         b  ...                        k                      b   
2            c         b  ...                        s                      w   
3            c         b  ...                        s                      w   
4            c         b  ...                        f                      w   

  stalk-color-below-ring veil-type veil-color ring-number ring-type  \
0                      w         p          w           o         e   
1                      p         p          w           o         l   
2                      w         p          w           o         p   
3                      w         p          w           o         p   
4                      w         p          w           o         p   

  spore-print-color population habitat  
0                 w          v       d  
1                 h          y       g  
2                 k          n       m  
3                 h          v       u  
4                 h          v       g  

[5 rows x 24 columns]

In [5]:
train.shape

(4062, 24)

In [6]:
train.describe(include="O")

Y cap-shape cap-surface cap-color bruises  odor gill-attachment  \
count   4062      4062        4062      4062    4062  4062            4062   
unique     2         6           4        10       2     9               2   
top        e         x           y         n       f     n               f   
freq    2103      1809        1653      1166    2367  1747            3964   

       gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
count          4062      4062       4062  ...                     4062   
unique            2         2         12  ...                        4   
top               c         b          b  ...                        s   
freq           3404      2796        871  ...                     2463   

       stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
count                    4062                   4062      4062       4062   
unique                      9                      9         1          4   
top                         w                      w         p          w   
freq                     2281                   2213      4062       3968   

       ring-number ring-type spore-print-color population habitat  
count         4062      4062              4062       4062    4062  
unique           3         5                 9          6       7  
top              o         p                 w          v       d  
freq          3726      1977              1224       2005    1562  

[4 rows x 23 columns]

In [7]:
train.isnull().sum()

id                          0
Y                           0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [8]:
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import GridSearchCV
import pydotplus

clf = DT()

In [9]:
parameters = {"max_depth": list(range(2,11)),"min_samples_leaf": [5,10,20,50,100,500]}

In [10]:
trainX = train.iloc[:, 2:25]
y = train["Y"]

In [11]:
trainX = pd.get_dummies(trainX)

In [12]:
y = y.replace({"p":0, "e":1})

In [13]:
gcv = GridSearchCV(clf, parameters, cv=5, scoring="roc_auc", n_jobs=-1, return_train_score=True)
gcv.fit(trainX, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [5, 10, 20, 50, 100, 500]},
             return_train_score=True, scoring='roc_auc')

In [14]:
gcv.best_params_

{'max_depth': 7, 'min_samples_leaf': 5}

In [15]:
pred = gcv.predict(trainX)

In [16]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y, pred)
print("AUC=", metrics.auc(fpr, tpr))

AUC= 1.0


In [17]:
test.isnull().sum()

id                          0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [18]:
test.shape

(4062, 23)

In [19]:
testX = test.iloc[:,1:24]
testX = pd.get_dummies(testX)

In [20]:
pred_test = gcv.predict(testX)

In [50]:
sample.head()

0  1
0  1  e
1  4  e
2  6  e
3  8  p
4  9  e

In [51]:
sample[1] = pred_test

In [52]:
sample[1].replace({0: "p", 1:"e"}, inplace=True)

In [53]:
sample.head()

0  1
0  1  p
1  4  e
2  6  e
3  8  p
4  9  p

In [54]:
sample.to_csv("submit01.csv", index=None, header=None)